## Importación de Librerías

In [ ]:
# !pip install iso639

In [181]:
import os
import os.path
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import ast
from ast import literal_eval
import iso639

print('Importanción completa')

Importancion completa


# **Ingesta de Datos**

Las siguientes líneas de código nos permitirán acceder a los archivos que se encuentran en la carpeta Dataset

In [4]:
# Conocer que extensiones se encuentran en la carpeta Dataset
dataset_dir = '/content/drive/MyDrive/Colab Notebooks/PI_MLOps/Dataset'
files = os.listdir(dataset_dir)

for archivo in files:
    nombre, extension = os.path.splitext(archivo)
    print("Extensión '{}'".format(
  extension))

Extensión '.csv'
Extensión '.csv'


## Archivos .csv

In [5]:
# Creando una lista con el path de los archivos detectados en la carpeta files
csv_files = []

for file in files:
    if os.path.isfile(os.path.join(dataset_dir, file)) and file.endswith('.csv'):
        csv_files.append(f'{dataset_dir}/{file}')

print(f'csv_files: {csv_files}')

csv_files: ['/content/drive/MyDrive/Colab Notebooks/PI_MLOps/Dataset/credits.csv', '/content/drive/MyDrive/Colab Notebooks/PI_MLOps/Dataset/movies_dataset.csv']


In [6]:
credits_csv = pd.read_csv(csv_files[0])
movies_csv = pd.read_csv(csv_files[1], low_memory = False)  # As 10 columns have mixed types, set low_memory as parameter.

print('archivos .csv leidos correctamente')

archivos .csv leidos correctamente


# **Tratamiento de datos**

Acontinuación se transformará y limpiará la data de cada archivo .csv por separado.

## **credits_csv**

### Informacion de credits_csv

In [7]:
# Retornar las primeras 5 filas del df
credits_csv.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [8]:
# Retornar las ultimas 5 filas del df
credits_csv.tail()

,cast,crew,id
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...",439050
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758
45474,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",227506
45475,[],"[{'credit_id': '593e676c92514105b702e68e', 'de...",461257


In [9]:
# Informacion del número de columnas, filas, tipos de datos y valores faltantes
credits_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [10]:
# Cantidad de registros y columnas respectivamente
entries_original = credits_csv.shape[0]
credits_csv.shape

(45476, 3)

### Datos faltantes en la data cruda
¿cuántos datos faltantes hay y que % representan en base al total de datos?

In [11]:
# Cantidad de valores faltantes por columna
missing_values_count = credits_csv.isnull().sum()
columns = credits_csv.shape[1]
missing_values_count[0:columns]

cast    0
crew    0
id      0
dtype: int64

### Examinando la data en formato JSON por atributo

In [12]:
# Acceder a los datos de la primera fila del df
credits_csv.loc[0]

cast    [{'cast_id': 14, 'character': 'Woody (voice)',...
crew    [{'credit_id': '52fe4284c3a36847f8024f49', 'de...
id                                                    862
Name: 0, dtype: object

Hay 2 columnas en formato JSON (cast, crew)

In [13]:
# Acceder a los datos de la fila 390 de la columna crew
credits_csv.crew.loc[390]

"[{'credit_id': '535e69b80e0a264fdb006416', 'department': 'Directing', 'gender': 2, 'id': 126837, 'job': 'Director', 'name': 'Michael A. Nickles', 'profile_path': None}, {'credit_id': '535e69c60e0a264fe80065ef', 'department': 'Writing', 'gender': 2, 'id': 126837, 'job': 'Writer', 'name': 'Michael A. Nickles', 'profile_path': None}]"

In [14]:
# Acceder a los datos de la fila 6402 de la columna crew
credits_csv.cast.loc[6402]

"[{'cast_id': 1, 'character': 'Puck', 'credit_id': '52fe43e6c3a368484e004da9', 'gender': 2, 'id': 54455, 'name': 'Devon Gummersall', 'order': 0, 'profile_path': '/9mw9fqm1dtxObQbG6lUGvlvXvt7.jpg'}, {'cast_id': 2, 'character': 'Johnny Black', 'credit_id': '52fe43e6c3a368484e004dad', 'gender': 2, 'id': 52476, 'name': 'Dylan Bruno', 'order': 1, 'profile_path': '/4JdgfAHPodXaJEHNeogxboQtMZp.jpg'}, {'cast_id': 3, 'character': 'Karla', 'credit_id': '52fe43e6c3a368484e004db1', 'gender': 1, 'id': 56356, 'name': 'Gina Philips', 'order': 2, 'profile_path': '/pj9O7yhpZ9JA0fP4bFu3l1xFKhl.jpg'}, {'cast_id': 4, 'character': 'Sweeney', 'credit_id': '52fe43e7c3a368484e004db5', 'gender': 2, 'id': 73589, 'name': 'Johnny Whitworth', 'order': 3, 'profile_path': '/xkeywGnloZXCsHjKz6eqbKoClrh.jpg'}, {'cast_id': 8, 'character': 'Jody', 'credit_id': '52fe43e7c3a368484e004dcb', 'gender': 0, 'id': 15012, 'name': 'Katharine Towne', 'order': 4, 'profile_path': '/33AK0lDtxIhh7125JC7daDWMSkC.jpg'}, {'cast_id': 10, 

### Buscando inconsistencias en la entrada de los datos (Data Entry)

#### Atributo id

In [15]:
# Retonar un array con los datos numericos
unique_id = credits_csv.id.unique()
unique_id.sort()
print(unique_id)

[     2      3      5 ... 468343 468707 469172]


In [16]:
# Cantidad de veces que se repiten los valores numericos
credits_csv.id.value_counts().head()

141971    3
298721    2
9755      2
10991     2
99080     2
Name: id, dtype: int64

Hay id duplicados. Antes de tomar una decision respecto a ello se procedera primero a desanidar la data, extraer la informacion necesaria y finalmente examinar estos registros duplicados.

#### Atributos cast y crew

In [17]:
filtro_cast = credits_csv[credits_csv['cast'].str.contains('[]', regex = False)]
filtro_cast.head()

,cast,crew,id
137,[],"[{'credit_id': '52fe4ab0c3a368484e161d3d', 'de...",124639
240,[],"[{'credit_id': '52fe464ac3a36847f80f6d61', 'de...",43475
393,[],"[{'credit_id': '52fe4624c3a36847f80ef0a5', 'de...",42981
438,[],"[{'credit_id': '52fe448dc3a368484e029383', 'de...",24257
595,[],"[{'credit_id': '52fe4aacc3a368484e16115b', 'de...",124472


In [18]:
empty_data_cast = filtro_cast.cast.shape[0]
print(f'Hay {empty_data_cast} registros en la columna cast con strings que contienen una lista vacia "[]"')

Hay 2418 registros en la columna cast con strings que contienen una lista vacia "[]"


In [19]:
filtro_crew = credits_csv[credits_csv['crew'].str.contains('[]', regex = False)]
filtro_crew.head()

,cast,crew,id
189,"[{'cast_id': 4, 'character': 'Himself', 'credi...",[],56088
614,"[{'cast_id': 1, 'character': 'Grace Rhodes', '...",[],123505
635,"[{'cast_id': 0, 'character': 'Joachim Krippo',...",[],339428
661,[],[],318177
711,[],[],365371


In [20]:
empty_data_crew = filtro_crew.crew.shape[0]
print(f'Hay {empty_data_crew} registros en la columna crew con strings que contienen una lista vacia "[]"')

Hay 771 registros en la columna crew con strings que contienen una lista vacia "[]"


**Observacion:**

Hay que tomar en cuenta que todos estos datos ```'[]'``` representan valores NaN, por lo que las variables ```empty_data_cast``` y ```empty_data_crew``` se utilizaran mas adelante para comparar los valores NaN resultantes de la data limpia.

### Desaninando la Data

Como los datos se presentan en forma de listas "stringificadas" ("stringified" lists) se procede a tratarlos, con el objetivo de que obtengan una estructura manejable para asi poder extraer la información que necesitamos.

In [21]:
# Copia para no afectar los datos originales
copy_credits = credits_csv.copy()

In [22]:
# Convertir una cadena que contiene una lista en una lista de objetos
columns_credits = ['cast', 'crew']

for atribute in columns_credits:
  copy_credits[atribute] = copy_credits[atribute].apply(lambda x: ast.literal_eval(str(x)))

copy_credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': 'Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [23]:
# Visualizar las key-values que se encuentran en el atributo cast
copy_credits.cast.loc[0][0]

{'cast_id': 14,
 'character': 'Woody (voice)',
 'credit_id': '52fe4284c3a36847f8024f95',
 'gender': 2,
 'id': 31,
 'name': 'Tom Hanks',
 'order': 0,
 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}

In [24]:
# Visualizar las key-values que se encuentran en el atributo crew
copy_credits.crew.loc[0][0]

{'credit_id': '52fe4284c3a36847f8024f49',
 'department': 'Directing',
 'gender': 2,
 'id': 7879,
 'job': 'Director',
 'name': 'John Lasseter',
 'profile_path': '/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg'}

In [25]:
# Profesiones que se encuentran en la clave job del atributo crew
pd.DataFrame(copy_credits.crew[0])['job'].value_counts().sort_index().head(12)

ADR Editor                 1
ADR Voice Casting          1
Animation                 26
Animation Director         2
Art Direction              1
Assistant Editor           4
Assistant Sound Editor     4
CG Painter                 2
Casting Consultant         1
Character Designer         8
Color Timer                1
Director                   1
Name: job, dtype: int64

Del atributo cast se extraerán los nombres de los actores de las peliculas, mientras que para el caso del atributo crew se extraerá el nombre de los directores de cada película.

#### Columna crew

In [26]:
# Funcion que extrae y retorna el nombre del director en el atributo crew
def get_director(data):
  # Verifica si data es una lista
  if isinstance(data, list):
    # Verifica si la lista data esta vacia
    if len(data) == 0:
      return np.nan

    # Crea una lista de nombres de directores que cumplen la condición
    director_names = [i['name'] for i in data if i['job'] == 'Director']
    # Verifica si la lista dictor_names esta vacia
    if len(director_names) == 0:
      return np.nan
    # Une los nombres de los directores con una coma y un espacio
    return ','.join(director_names)
  else:
    # Si data no es una lista, devuelve NaN
    return np.nan

print('Función creada')

Función creada


In [27]:
# Aplicar get_director a cada elemento de la columna "crew"
copy_credits['director'] = copy_credits['crew'].apply(lambda x: get_director(x))
copy_credits.head()

,cast,crew,id,director
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,John Lasseter
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844,Joe Johnston
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602,Howard Deutch
3,"[{'cast_id': 1, 'character': 'Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357,Forest Whitaker
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862,Charles Shyer


In [28]:
# Verificar si hay cadenas vacios
copy_credits[copy_credits['director'] == '']

,cast,crew,id,director


In [29]:
# Cantidad de veces que se repiten los elementos extraidos
copy_credits['director'].value_counts(dropna = False).head(5)

NaN                 887
John Ford            63
Michael Curtiz       61
Alfred Hitchcock     52
Werner Herzog        52
Name: director, dtype: int64

#### Columna cast

In [30]:
# Función que extrae y returna los nombres de los actores que se encuentran en el atributo cast
def get_actors(data):
  # Verifica si data es una lista
  if isinstance(data, list) == True:
    # Verifica si la lista data esta vacia
    if len(data) == 0:
      return np.nan

    # Crea una lista de nombres de actores
    actors_names = [i['name'] for i in data]
    # Verifica si la lista actors_names esta vacia
    if len(actors_names) == 0:
      return np.nan
    # Une los nombres de los directores con una coma y un espacio
    return ','.join(actors_names)
  else:
    # Si data no es una lista, devuelve NaN
    return np.nan

print('Función creada')

Función creada


In [31]:
# Aplicar get_actors a cada elemento de la columna "crew"
copy_credits['actors'] = copy_credits['cast'].apply(lambda x: get_actors(x))
copy_credits.head()

,cast,crew,id,director,actors
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,John Lasseter,"Tom Hanks,Tim Allen,Don Rickles,Jim Varney,Wal..."
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844,Joe Johnston,"Robin Williams,Jonathan Hyde,Kirsten Dunst,Bra..."
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602,Howard Deutch,"Walter Matthau,Jack Lemmon,Ann-Margret,Sophia ..."
3,"[{'cast_id': 1, 'character': 'Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357,Forest Whitaker,"Whitney Houston,Angela Bassett,Loretta Devine,..."
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862,Charles Shyer,"Steve Martin,Diane Keaton,Martin Short,Kimberl..."


In [32]:
# Verificar si hay cadenas vacias
copy_credits[copy_credits['actors'] == '']

,cast,crew,id,director,actors


In [33]:
# # Cantidad de veces que se repiten los elementos extraidos
copy_credits['actors'].value_counts(dropna = False).head()

NaN               2418
Georges Méliès      24
Louis Theroux       15
Mel Blanc           12
Jimmy Carr           9
Name: actors, dtype: int64

Los atributos que seran utilizados posteriormente seran: id, director y actors

In [34]:
# Dataframe con los datos desanidados
credits = copy_credits[['id', 'director', 'actors']].copy()
credits.head()

,id,director,actors
0,862,John Lasseter,"Tom Hanks,Tim Allen,Don Rickles,Jim Varney,Wal..."
1,8844,Joe Johnston,"Robin Williams,Jonathan Hyde,Kirsten Dunst,Bra..."
2,15602,Howard Deutch,"Walter Matthau,Jack Lemmon,Ann-Margret,Sophia ..."
3,31357,Forest Whitaker,"Whitney Houston,Angela Bassett,Loretta Devine,..."
4,11862,Charles Shyer,"Steve Martin,Diane Keaton,Martin Short,Kimberl..."


### Valores Faltantes con la data desanidada

Verificando valores nulos y comparando con los datos contenidos en las variables ```empty_data_cast``` y ```empty_data_crew```

In [35]:
# Cantidad de valores faltantes por columna
missing_values_count = credits.isnull().sum()
columns = credits.shape[1]
missing_values_count[0:columns]

id             0
director     887
actors      2418
dtype: int64

In [36]:
# Comparativa de los NaN con '[]' en crew
data_nan_director = credits['director'].isnull().sum()
print('Columna crew')
print(f'Cantidad de registros del tipo string con listas vacias contenidas ("[]"): {empty_data_crew}')
print(f'Cantidad de registros NaN: {data_nan_director}')

Columna crew
Cantidad de registros del tipo string con listas vacias contenidas ("[]"): 771
Cantidad de registros NaN: 887


Se observa que hubo un aumento en los valores nulos (887) y esto se debe a que ademas de las listas vacias contenidas en strings, en las listas no vacias 116 de ellas no tenian en los diccionarios el nombre del Director

In [37]:
# Comparativa de los NaN con '[]' en actors
data_nan_actors = credits['actors'].isnull().sum()
print('Columna cast')
print(f'Cantidad de registros del tipo string con listas vacias contenidas ("[]"): {empty_data_cast}')
print(f'Cantidad de registros NaN: {data_nan_actors}')

Columna cast
Cantidad de registros del tipo string con listas vacias contenidas ("[]"): 2418
Cantidad de registros NaN: 2418


La cantidad de listas vacias contenidas en strings coincide con la cantidad de datos NaN, lo que indica que en todas las listas no vacias se encontro el valor correspondiente para los nombres de los actores.

**Observacion:**

Por ahora, no se tomara ninguna decision con respecto a los datos nulos dado que a continuacion se procede a la limpieza de los datos en el dataframe movies_csv de manera de poder realizar al final un merge() entre los dos dataframes (credicts con movies) y a partir de ahi se decidira que accion tomar referente a la data faltante.

### Comprobando y removiendo datos duplicados

In [38]:
# Verificar si hay datos duplicados en la columna id
credits[credits.duplicated(subset = 'id', keep = False)].shape[0]  # shape[0] numero de filas

87

In [39]:
# Los 5 id que mas se repiten
credits['id'].value_counts().head()

141971    3
298721    2
9755      2
10991     2
99080     2
Name: id, dtype: int64

In [40]:
# Verificar las 3 apariciones del id = 141971
credits[credits['id'] == 141971]

,id,director,actors
13261,141971,JP Siili,"Petteri Summanen,Ismo Kallio,Eppu Salminen,Iri..."
13375,141971,JP Siili,"Petteri Summanen,Ismo Kallio,Eppu Salminen,Iri..."
16764,141971,JP Siili,"Petteri Summanen,Ismo Kallio,Eppu Salminen,Iri..."


In [41]:
print(credits.iat[0,2])
print(credits.iat[1,2])
print(credits.iat[2,2])

Tom Hanks,Tim Allen,Don Rickles,Jim Varney,Wallace Shawn,John Ratzenberger,Annie Potts,John Morris,Erik von Detten,Laurie Metcalf,R. Lee Ermey,Sarah Freeman,Penn Jillette
Robin Williams,Jonathan Hyde,Kirsten Dunst,Bradley Pierce,Bonnie Hunt,Bebe Neuwirth,David Alan Grier,Patricia Clarkson,Adam Hann-Byrd,Laura Bell Bundy,James Handy,Gillian Barber,Brandon Obray,Cyrus Thiedeke,Gary Joseph Thorup,Leonard Zola,Lloyd Berry,Malcolm Stewart,Annabel Kershaw,Darryl Henriques,Robyn Driscoll,Peter Bryant,Sarah Gilson,Florica Vlad,June Lion,Brenda Lockmuller
Walter Matthau,Jack Lemmon,Ann-Margret,Sophia Loren,Daryl Hannah,Burgess Meredith,Kevin Pollak


Hay 87 id duplicados, los cuales se procede a eliminar

In [42]:
# Eliminando los id duplicados
credits.drop_duplicates(subset = 'id', inplace = True)

In [43]:
# Comprobando que ya no ha id duplicados
credits[credits.duplicated(subset = 'id', keep = False)].shape[0]

0

In [44]:
# Verificando que el id = 141971 ya no se repite 3 veces
credits[credits['id'] == 141971]

,id,director,actors
13261,141971,JP Siili,"Petteri Summanen,Ismo Kallio,Eppu Salminen,Iri..."


In [45]:
print(f'Cantidad de registros con duplicados: {entries_original}')
entries_final = credits.shape[0]
print(f'Cantidad de registros sin duplicados: {entries_final}')
print(f'% de datos eliminados respecto a la cantidad orginal: {round(100 - (entries_final * 100/entries_original), 4)}')

Cantidad de registros con duplicados: 45476
Cantidad de registros sin duplicados: 45432
% de datos eliminados respecto a la cantidad orginal: 0.0968


Finalmente:

In [46]:
credits.head()

,id,director,actors
0,862,John Lasseter,"Tom Hanks,Tim Allen,Don Rickles,Jim Varney,Wal..."
1,8844,Joe Johnston,"Robin Williams,Jonathan Hyde,Kirsten Dunst,Bra..."
2,15602,Howard Deutch,"Walter Matthau,Jack Lemmon,Ann-Margret,Sophia ..."
3,31357,Forest Whitaker,"Whitney Houston,Angela Bassett,Loretta Devine,..."
4,11862,Charles Shyer,"Steve Martin,Diane Keaton,Martin Short,Kimberl..."


## **movies_csv**

### Informacion de movies_csv

In [47]:
# Retornar las primeras 5 filas del df
movies_csv.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [48]:
# Retornar las ultimas 5 filas del df
movies_csv.tail()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0
45465,False,NaN,0,[],NaN,461257,tt6980792,en,Queerama,50 years after decriminalisation of homosexual...,...,2017-06-09,0.0,75.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Queerama,False,0.0,0.0


In [49]:
# Informacion respecto al número de columnas, filas, tipos de datos y valores faltantes
movies_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [50]:
# Cantidad de filas y columnas respectivamente
entries_original = movies_csv.shape[0]
movies_csv.shape

(45466, 24)

### Datos faltantes en la data cruda

¿cuántos datos faltantes hay y que % representan en base al total de datos?

In [51]:
# Cantidad de datos faltantes por columna
missing_values_count = movies_csv.isnull().sum()
columns = movies_csv.shape[1]
missing_values_count[0:columns]

adult                        0
belongs_to_collection    40972
budget                       0
genres                       0
homepage                 37684
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
video                        6
vote_average                 6
vote_count                   6
dtype: int64

In [52]:
# Porcentaje de datos faltantes respecto al total de registros
total_cells = np.product(movies_csv.shape)
total_missing = missing_values_count.sum()
percent_missing = (total_missing / total_cells) * 100
print(percent_missing, 2)

9.674078798809367 2


### Eliminando atributos

Dado que las columnas ```adult```, ```imdb_id```, ```homepage```, ```original_title```, ```poster_path```, ```video``` no serán utilizadas, se procede a eliminarlas.

In [53]:
columns_drop = ['adult', 'imdb_id', 'homepage', 'original_title', 'poster_path', 'video']
movies_csv.drop(columns_drop, axis = 1, inplace = True)
movies_csv.columns

Index(['belongs_to_collection', 'budget', 'genres', 'id', 'original_language',
       'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

### Examinando la data en formato JSON por atributo

In [54]:
movies_csv.loc[0]

belongs_to_collection    {'id': 10194, 'name': 'Toy Story Collection', ...
budget                                                            30000000
genres                   [{'id': 16, 'name': 'Animation'}, {'id': 35, '...
id                                                                     862
original_language                                                       en
overview                 Led by Woody, Andy's toys live happily in his ...
popularity                                                       21.946943
production_companies        [{'name': 'Pixar Animation Studios', 'id': 3}]
production_countries     [{'iso_3166_1': 'US', 'name': 'United States o...
release_date                                                    1995-10-30
revenue                                                        373554033.0
runtime                                                               81.0
spoken_languages                  [{'iso_639_1': 'en', 'name': 'English'}]
status                   

Hay cinco columnas en formato JSON (belongs_to_collection, genres, production_companies, production_countries, and spoken_languages).

In [55]:
movies_csv.belongs_to_collection.loc[0]

"{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}"

In [56]:
movies_csv.genres.loc[0]

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [57]:
movies_csv.production_companies.loc[0]

"[{'name': 'Pixar Animation Studios', 'id': 3}]"

In [58]:
movies_csv.production_countries.loc[0]

"[{'iso_3166_1': 'US', 'name': 'United States of America'}]"

In [59]:
movies_csv.spoken_languages.loc[0]

"[{'iso_639_1': 'en', 'name': 'English'}]"

### Buscando inconsistencias en la data en formato JSON

#### Atributo belongs_to_collection

In [60]:
# Verificando si hay registros del tipo strings con diccionarios contenidos vacios
filtro_btc = movies_csv[movies_csv['belongs_to_collection'].str.contains('{}', na = False, regex = False)]
filtro_btc

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count


#### Atributo genres

In [61]:
# Verificando si hay registros del tipo strings con listas contenidas vacias
filtro_genres = movies_csv[movies_csv['genres'].str.contains('[]', regex = False)]
filtro_genres.genres.head()

55     []
83     []
126    []
137    []
390    []
Name: genres, dtype: object

In [62]:
empty_data_genres = filtro_genres.genres.shape[0]
print(f'Hay {empty_data_genres} registros en la columna genres con strings que contienen una lista vacia "[]"')

Hay 2442 registros en la columna genres con strings que contienen una lista vacia "[]"


#### Atributo production_companies

In [63]:
# Verificando si hay registros del tipo strings con listas contenidas vacias
filtro_pc = movies_csv[movies_csv['production_companies'].str.contains('[]', na = False, regex = False)]
filtro_pc.production_companies.head()

50    []
52    []
57    []
58    []
83    []
Name: production_companies, dtype: object

In [64]:
empty_data_pc = filtro_pc['production_companies'].shape[0]
print(f'Hay {empty_data_pc} registros en la columna production_companies con strings que contienen una lista vacia "[]"')

Hay 11875 registros en la columna production_companies con strings que contienen una lista vacia "[]"


#### Atributo production_countries

In [65]:
# Verificando si hay registros del tipo strings con listas contenidas vacias
filtro_countries = movies_csv[movies_csv['production_countries'].str.contains('[]', na = False, regex = False)]
filtro_countries.production_countries.head()

50     []
55     []
83     []
106    []
107    []
Name: production_countries, dtype: object

In [66]:
empty_data_countries = filtro_countries['production_countries'].shape[0]
print(f'Hay {empty_data_countries} registros en la columna production_countries con strings que contienen una lista vacia "[]"')

Hay 6282 registros en la columna production_countries con strings que contienen una lista vacia "[]"


#### Atributo spoken_languages

In [67]:
# Verificando si hay registros del tipo strings con listas contenidas vacias
filtro_languages = movies_csv[movies_csv['spoken_languages'].str.contains('[]', na = False, regex = False)]
filtro_languages.spoken_languages.head()

50     []
83     []
107    []
126    []
137    []
Name: spoken_languages, dtype: object

In [68]:
empty_data_languages = filtro_languages['spoken_languages'].shape[0]
print(f'Hay {empty_data_languages} registros en la columna spoken_languages con strings que contienen una lista vacia "[]"')

Hay 3829 registros en la columna spoken_languages con strings que contienen una lista vacia "[]"


### Desinando la data

Al igual que en el dataset credits_csv, hay atributos donde los datos se presentan en forma de listas "stringificadas" ("stringified" lists) por lo que de la misma forma se procede a tratarlos, con el objetivo de que obtengan una estructura manejable para poder extraer la información que necesitamos.

In [69]:
# Copia para no afectar los datos originales
copy_movies = movies_csv.copy()

In [70]:
# Funcion que retorna el literal de python que se encuentre contenido en cadenas
def stringified(data):
  # Verifica si data es un string
  if isinstance(data, str):
    # Evaluar cadenas que contienen literales y devuelve el objeto correspondiente (para este caso lista o dict)
    x = ast.literal_eval(data)
    return x
  else:
    # Si data no es un string, devuelve NaN
    return np.nan

print('Function created')

Function created


In [71]:
# Columnas con datos del tipo string que contienen literales de python
columns_movies = ['belongs_to_collection', 'genres', 'production_companies', 'production_countries', 'spoken_languages']

for columns in columns_movies:
  # Aplicar la funcion stringified() a cada columna de columns_movies
  copy_movies[columns] = copy_movies[columns].apply(lambda x: stringified(x))

copy_movies.head(3)

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0
1,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,A family wedding reignites the ancient feud be...,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0


Como la informacion que nos interesa extraer de los datos anidados es el valor de la clave nombre para cada atributo. Se procede a extraerlos

#### Atributo belongs_to_collection

In [72]:
# Funcion que extrae y returna un string con el valor de la clave name para cada atributo
def get_name(data):
  # Verifica si data es un diccionario
  if isinstance(data, dict):
    # Alamacena el valor de la clave name
    name = data['name']
    # Verifica si el valor es un string vacio
    if name == '':
      return np.nan
    return name
  else:
    # Si data no es un dict, devuelve NaN
    return np.nan

print('Functions created')

Functions created


In [73]:
# Aplicar get_name para el atributo belongs_to_collection dado que sus registros son del tipo dict
copy_movies['collection'] = copy_movies['belongs_to_collection'].apply(lambda x: get_name(x))
copy_movies['collection'].head()

0              Toy Story Collection
1                               NaN
2         Grumpy Old Men Collection
3                               NaN
4    Father of the Bride Collection
Name: collection, dtype: object

In [74]:
copy_movies['collection'].value_counts(dropna = False).head()

NaN                              40975
The Bowery Boys                     29
Totò Collection                     27
James Bond Collection               26
Zatôichi: The Blind Swordsman       26
Name: collection, dtype: int64

#### Atributos genres, production_companies, production_countries, spoken_languages

In [75]:
# Función que extrae y returna el valor de las claves 'name'
def get_names(data):
  # Verifica si data es una lista
  if isinstance(data, list):
    # Verifica si la lista data esta vacia
    if len(data) == 0:
      return np.nan

    # Crea una lista con los valores obtenidos
    names = [i['name'] for i in data]
    # Verifica si la lista names esta vacia
    if len(names) == 0:
      return np.nan
    # Une los nombres extraidos con una coma y un espacio
    return ','.join(names)
  else:
    # Si data no es una lista, devuelve NaN
    return np.nan

print('Functions created')

Functions created


In [76]:
# Atributos que sus registros son del tipo lista
columns_movies_1 = ['genres', 'production_companies', 'production_countries']

for atribute in columns_movies_1:
  # Aplicar la funcion get_names() a cada columna de columns_movies_1
  copy_movies[atribute] = copy_movies[atribute].apply(lambda x: get_names(x))

# Aplicar la funcion get_names() al atributo "spoken_languages" y almacenar en otra variable llamada "languages"
copy_movies['languages'] = copy_movies['spoken_languages'].apply(lambda x: get_names(x))

In [77]:
# Confirmar que ya no hay registros anidados y se extrajo la informacion solucitada
copy_movies[['collection', 'genres', 'production_companies', 'production_countries', 'languages']].head()

,collection,genres,production_companies,production_countries,languages
0,Toy Story Collection,"Animation,Comedy,Family",Pixar Animation Studios,United States of America,English
1,NaN,"Adventure,Fantasy,Family","TriStar Pictures,Teitler Film,Interscope Commu...",United States of America,"English,Français"
2,Grumpy Old Men Collection,"Romance,Comedy","Warner Bros.,Lancaster Gate",United States of America,English
3,NaN,"Comedy,Drama,Romance",Twentieth Century Fox Film Corporation,United States of America,English
4,Father of the Bride Collection,Comedy,"Sandollar Productions,Touchstone Pictures",United States of America,English


**Atrbiuto genres**

In [78]:
# Los primeros 5 valores que mas se repiten en la columna
copy_movies['genres'].value_counts(dropna = False).head()

Drama            5000
Comedy           3621
Documentary      2723
NaN              2442
Drama,Romance    1301
Name: genres, dtype: int64

In [79]:
# Comparacion entre los valores '[]' y NaN en la columna genres
empty_data_nan = copy_movies['genres'].isnull().sum()
print(f'Hay {empty_data_nan} datos nulos en la columna genres')
print(f'Habian {empty_data_genres} datos con listas vacias contenidas en un string ("[]")')

Hay 2442 datos nulos en la columna genres
Habian 2442 datos con listas vacias contenidas en un string ("[]")


In [80]:
# Verificar si retornaron strings vacios
copy_movies[copy_movies['genres'] == '']

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,collection,languages


**Atributo production_companies**

In [81]:
# Los primeros 5 valores que mas se repiten en la columna production_companies
copy_movies['production_companies'].value_counts(dropna = False).head()

NaN                                       11881
Metro-Goldwyn-Mayer (MGM)                   742
Warner Bros.                                540
Paramount Pictures                          505
Twentieth Century Fox Film Corporation      439
Name: production_companies, dtype: int64

In [82]:
# Comparacion entre los valores '[]' y NaN en la columna production_companies
empty_data_nan = copy_movies['production_companies'].isnull().sum()
print(f'Hay {empty_data_nan} datos nulos en la columna production_companies')
print(f'Habian {empty_data_pc} datos con listas vacias contenidas en un string ("[]")')

Hay 11881 datos nulos en la columna production_companies
Habian 11875 datos con listas vacias contenidas en un string ("[]")


La cantidad de datos nulos en el atributo production_companies aumento debido a que en las listas no vacias hay diccionarios que no tienen el nombre de la compania de produccion

In [83]:
# Verificar si retornaron strings vacios
copy_movies[copy_movies['production_companies'] == '']

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,collection,languages


**Atributo production_countries**

In [84]:
# Los primeros 5 valores que mas se repiten en la columna production_countries
copy_movies['production_countries'].value_counts(dropna = False).head()

United States of America    17851
NaN                          6288
United Kingdom               2238
France                       1654
Japan                        1356
Name: production_countries, dtype: int64

In [85]:
# Comparacion entre los valores '[]' y NaN en la columna production_countries
empty_data_nan = copy_movies['production_countries'].isnull().sum()
print(f'Hay {empty_data_nan} datos nulos en la columna production_countries')
print(f'Habian {empty_data_countries} datos con listas vacias contenidas en un string ("[]")')

Hay 6288 datos nulos en la columna production_countries
Habian 6282 datos con listas vacias contenidas en un string ("[]")


La cantidad de datos nulos en el atributo production_countries aumento debido a que en las listas no vacias hay diccionarios que no tienen el nombre de los paises productores

In [86]:
# Verificar si retornaron strings vacios
copy_movies[copy_movies['production_countries'] == '']

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,collection,languages


**Atributes languages**

In [87]:
# Los primeros 5 valores que mas se repiten en la columna languages
copy_movies['languages'].value_counts(dropna = False)

English                              22395
NaN                                   3835
Français                              1853
日本語                                   1289
Italiano                              1218
                                     ...  
Français,Latin,,Português,English        1
Español,עִבְרִית,English                 1
English,Íslenska,Pусский                 1
فارسی,                                   1
Fulfulde,English                         1
Name: languages, Length: 1843, dtype: int64

In [88]:
# Comparacion entre los valores '[]' y NaN en la columna languages
empty_data_nan = copy_movies['languages'].isnull().sum()
print(f'Hay {empty_data_nan} datos nulos en la columna languages')
print(f'Habian {empty_data_languages} datos con listas vacias contenidas en un string ("[]")')

Hay 3835 datos nulos en la columna languages
Habian 3829 datos con listas vacias contenidas en un string ("[]")


La cantidad de datos nulos en el atributo spoken_languages aumento debido a que en las listas no vacias hay diccionarios que no tienen el idioma hablado. Sin embargo, veamos la siguiente linea de codigo:

In [89]:
# Verificar si retornaron strings vacios
copy_movies[copy_movies['languages'] == ''].languages.head()

773     
4184    
5283    
7778    
7798    
Name: languages, dtype: object

In [90]:
movies_csv['spoken_languages'].loc[773]

"[{'iso_639_1': 'tl', 'name': ''}]"

Retornaron strings vacios dado que en los diccionarios para spoken_languages el valor de 'name' fue un string vacio, sin embargo, podemos ver que el diccionario tiene el iso639 correspondiente a esos lenguajes hablados. Por tanto, se crea una columna concerniente al iso_639_1 y esta la usaremos para imputar esos datos con strings vacios de la columna languages.

In [ ]:
# Función que extrae y returna el valor de las claves 'iso_639_1'
def get_iso639(data):
  # Verifica si data es una lista
  if isinstance(data, list):
    # Verifica si la lista data esta vacia
    if len(data) == 0:
      return np.nan

    # Crea una lista con los valores obtenidos
    iso639 = [i['iso_639_1'] for i in data]
    # Verifica si la lista iso639 esta vacia
    if len(iso639) == 0:
      return np.nan
    # Une los nombres extraidos con una coma y un espacio
    return ','.join(iso639)
  else:
    # Si data no es una lista, devuelve NaN
    return np.nan

print('Functions created')

In [ ]:
# Aplicar la funcion get_iso639() al atributo "spoken_languages" y almacenar en otra variable llamada "languages"
copy_movies['iso_639_1'] = copy_movies['spoken_languages'].apply(lambda x: get_iso639(x))

In [ ]:
# Crear un diccionario que mapee los códigos iso_639_1 a los nombres de idiomas
lang_dict = {lang.alpha2: lang.name for lang in iso639.languages}
# Funcion que reemplaza los valores faltantes en la columna de lenguages con los nombres de idiomas correspondientes
def replace_missing_lang(row):
    if row['languages'] == '':
        return lang_dict.get(row['iso_639_1'], '')
    else:
        return row['languages']

# Aplicar la función a la columna de lenguajes
copy_movies['languages'] = copy_movies.apply(replace_missing_lang, axis = 1)

In [ ]:
# Verificar si se aplicaron los cambios y no hay strings vacios
copy_movies[copy_movies['languages'] == ''].languages.head()

### Corrigiendo el tipo de dato

In [91]:
copy_movies.dtypes

belongs_to_collection     object
budget                    object
genres                    object
id                        object
original_language         object
overview                  object
popularity                object
production_companies      object
production_countries      object
release_date              object
revenue                  float64
runtime                  float64
spoken_languages          object
status                    object
tagline                   object
title                     object
vote_average             float64
vote_count               float64
collection                object
languages                 object
dtype: object

In [92]:
copy_movies.loc[1]

belongs_to_collection                                                  NaN
budget                                                            65000000
genres                                            Adventure,Fantasy,Family
id                                                                    8844
original_language                                                       en
overview                 When siblings Judy and Peter discover an encha...
popularity                                                       17.015539
production_companies     TriStar Pictures,Teitler Film,Interscope Commu...
production_countries                              United States of America
release_date                                                    1995-12-15
revenue                                                        262797249.0
runtime                                                              104.0
spoken_languages         [{'iso_639_1': 'en', 'name': 'English'}, {'iso...
status                   

De la informacio obtenida, se procede a cambiar el tipo de dato de los atributos:
- budget (de object a int)
- id (de object a int)
- popularity (de object a float)
- release_date (de object a datetime)

#### Convertiendo al tipo numérico

**Atributo id**

Pandas no pudo leerlo con el formato correcto, por lo que es un indicativo de que la columna posee algun dato no numerico.

Si se utiliza la funcion astype() para cambiar el tipo de dato de id a int, aparece un error ValueError: invalid literal for int() with base 10: '1997-08-20'. Por lo que se procede aplicar un filtro buscando todos los datos que tengan '-'

In [93]:
# Filtrar copy_movies para seleccionar las filas que contienen el carácter "-" en la columna "id"
copy_movies[copy_movies['id'].str.contains('-', regex = False)]

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,collection,languages
19730,0.065736,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"Carousel Productions,Vision View Entertainment...",1997-08-20,104.0,Released,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29503,1.931659,/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg,"Aniplex,GoHands,BROSTA TV,Mardock Scramble Pro...",2012-09-29,68.0,Released,NaN,NaN,NaN,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35587,2.185485,/zaSf5OG7V8X8gqFvly88zDdRm46.jpg,"Odyssey Media,Pulser Productions,Rogue State,T...",2014-01-01,82.0,Released,Beware Of Frost Bites,NaN,NaN,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Se observa que las filas donde el id es un string con un formato de tipo fecha, en los demas atributos (belongs_to_collection, budget, genres, original_language	overview	popularity, release_date) hay datos incorrectos. A simple vista pareciera que hay un desfase lo que deberia estar en un columna se encuentra en otra. Por otro lado, en el resto de los atributos (production_companies, production_countries, revenue, runtime, spoken_languages, status, tagline, title, vote_average, vote_count) hay datos nulos.

Dado que:
- La informacion que proporcinan es incorrecta para cada atributo,
- De 18 columnas 7 poseen datos basura y 11 datos nulos,
- No se conoce el id y tampoco el titulo de las peliculas.

Se decide eliminar las 3 filas. Considerando que la perdida de informacion relavante es casi nula comparando la cantidad de registros del dataset y que en estas 3 filas la mayoria de los datos son nulos.

In [94]:
# Eliminar las filas con índices 2, 4 y 6 del DataFrame copy_movies
copy_movies.drop(index = [19730, 29503, 35587], inplace = True)

In [95]:
# Verificando los cambios realizados
copy_movies[copy_movies['id'].str.contains('-', regex = False)]

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,collection,languages


In [96]:
# Se convierte la columna "id" de un tipo de datos object a un tipo de datos int32
copy_movies['id'] = copy_movies['id'].astype('int32', errors = 'raise')
copy_movies['id'].dtypes

dtype('int32')

**Atributo budget**

Como budget posee datos del tipo string numericos, se procede a filtrar las filas en busca de puntos o comas y saber si los datos son del tipo discreto o continuos

In [97]:
# Filtrar copy_movies para seleccionar las filas que contienen el carácter "." en la columna "budget"
copy_movies[copy_movies['budget'].str.contains('.', regex = False)]

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,collection,languages


In [98]:
# Filtrar copy_movies para seleccionar las filas que contienen el carácter "," en la columna "budget"
copy_movies[copy_movies['budget'].str.contains(',', regex = False)]

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,collection,languages


In [99]:
# Se convierte la columna "budget" de un tipo de datos object a un tipo de datos int32
copy_movies['budget'] = copy_movies['budget'].astype('int32', errors = 'raise')
copy_movies['budget'].dtypes

dtype('int32')

**Atributo popularity**

Como popularity posee datos del tipo string numericos, se procede a filtrar las filas en busca de puntos o comas y saber si los datos son del tipo discreto o continuo

In [100]:
# Filtrar copy_movies para seleccionar las filas que contienen el carácter "." en la columna "popularity"
filtro_popularity = copy_movies[copy_movies['popularity'].str.contains('.', na = False, regex = False)]
filtro_popularity.popularity.head()

0    21.946943
1    17.015539
2      11.7129
3     3.859495
4     8.387519
Name: popularity, dtype: object

In [101]:
# Se convierte la columna "popularity" de un tipo de datos object a un tipo de datos int32
copy_movies['popularity'] = copy_movies['popularity'].astype('float64', errors = 'raise')
copy_movies['popularity'].dtypes

dtype('float64')

#### Convertiendo al tipo datetime

**Atributo release_date**

In [102]:
# Verificando el formato de la fecha
copy_movies['release_date'].head()

0    1995-10-30
1    1995-12-15
2    1995-12-22
3    1995-12-22
4    1995-02-10
Name: release_date, dtype: object

In [103]:
copy_movies['release_date'] = pd.to_datetime(copy_movies['release_date'], format = '%Y-%m-%d')
copy_movies['release_date'].dtypes

dtype('<M8[ns]')

Validar si hay fechas mayores a la actual

In [104]:
# Obtener la fecha y hora actuales
current_datetime = dt.datetime.now()
# Convertir la fecha y hora actuales a un objeto datetime64[ns]
current_datetime64 = pd.to_datetime(current_datetime)
# Seleccionar las filas en el DataFrame copy_movies donde la columna "release_date" es mayor que la fecha y hora actuales
copy_movies[copy_movies['release_date'] > current_datetime64]

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,collection,languages


In [105]:
# Validar la ultima fecha en la se tomaron los datos (si estan actualizados)
copy_movies['release_date'].max()

Timestamp('2020-12-16 00:00:00')

In [106]:
# Verificando que los tipos de dato sean correctos
copy_movies.dtypes

belongs_to_collection            object
budget                            int32
genres                           object
id                                int32
original_language                object
overview                         object
popularity                      float64
production_companies             object
production_countries             object
release_date             datetime64[ns]
revenue                         float64
runtime                         float64
spoken_languages                 object
status                           object
tagline                          object
title                            object
vote_average                    float64
vote_count                      float64
collection                       object
languages                        object
dtype: object

### Verificando y removiendo datos duplicados

In [107]:
movies = copy_movies[['budget', 'genres',	'id',	'original_language', 'overview', 'popularity', 'production_companies',
                      'production_countries',	'release_date',	'revenue', 'runtime', 'status', 'tagline', 'title',	'vote_average',
                      'vote_count',	'collection',	'languages']].copy()

In [108]:
# Verificando la cantiad de duplicados en la columna id
movies[movies.duplicated(subset = 'id', keep = False)].shape[0]

59

In [109]:
frequent_ids = movies['id'].value_counts()[movies['id'].value_counts() >= 2]
frequent_ids.head()

141971    3
5511      2
132641    2
10991     2
168538    2
Name: id, dtype: int64

La siguiente linea de codigo muestra todos los id donde sus filas respecto a cada id se encuentran duplicadas. Estos id se encuentran contenidos en la lista id_duplicated

In [110]:
id_duplicated = [141971, 5511, 168538, 18440, 265189, 11115, 42495, 152795, 298721, 25541, 105045, 119916, 159849, 23305, 97995, 99080]
duplicated_movies = movies[movies['id'].isin(id_duplicated)].sort_values(by = 'id')
duplicated_movies.head(4)

,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count,collection,languages
7345,0,"Crime,Drama,Thriller",5511,fr,Hitman Jef Costello is a perfectionist who alw...,9.091288,"Fida cinematografica,Compagnie Industrielle et...","France,Italy",1967-10-25,39481.0,105.0,Released,There is no solitude greater than that of the ...,Le Samouraï,7.9,187.0,NaN,Français
9165,0,"Crime,Drama,Thriller",5511,fr,Hitman Jef Costello is a perfectionist who alw...,9.091288,"Fida cinematografica,Compagnie Industrielle et...","France,Italy",1967-10-25,39481.0,105.0,Released,There is no solitude greater than that of the ...,Le Samouraï,7.9,187.0,NaN,Français
24844,0,"Comedy,Drama",11115,en,As an ex-gambler teaches a hot-shot college ki...,6.880365,"Andertainment Group,Crescent City Pictures,Tag...",United States of America,2008-01-29,0.0,85.0,Released,NaN,Deal,5.2,22.0,NaN,English
14012,0,"Comedy,Drama",11115,en,As an ex-gambler teaches a hot-shot college ki...,6.880365,"Andertainment Group,Crescent City Pictures,Tag...",United States of America,2008-01-29,0.0,85.0,Released,NaN,Deal,5.2,22.0,NaN,English


In [111]:
# Encontrar y eliminar las filas duplicadas, se utiliza "inplace=True" para modificar directamente el objeto "movies"
for i in id_duplicated:
  index = movies[movies['id'] == i].index.tolist()
  if len(index) == 3:
    drop_index = index[-2:]
    movies.drop(labels = drop_index, inplace = True)
  else:
    drop_index = index[-1]
    movies.drop(labels = drop_index, inplace = True)

# Verificar que no hay id duplicados
movies[movies['id'].isin(id_duplicated)].sort_values(by = 'id').head()

,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count,collection,languages
7345,0,"Crime,Drama,Thriller",5511,fr,Hitman Jef Costello is a perfectionist who alw...,9.091288,"Fida cinematografica,Compagnie Industrielle et...","France,Italy",1967-10-25,39481.0,105.0,Released,There is no solitude greater than that of the ...,Le Samouraï,7.9,187.0,NaN,Français
14012,0,"Comedy,Drama",11115,en,As an ex-gambler teaches a hot-shot college ki...,6.880365,"Andertainment Group,Crescent City Pictures,Tag...",United States of America,2008-01-29,0.0,85.0,Released,NaN,Deal,5.2,22.0,NaN,English
14000,0,"Action,Horror,Science Fiction",18440,en,When a comet strikes Earth and kicks up a clou...,1.436085,NaN,United States of America,2007-01-01,0.0,89.0,Released,NaN,Days of Darkness,5.0,5.0,NaN,English
8068,0,"Adventure,Animation,Drama,Action,Foreign",23305,en,"In feudal India, a warrior (Khan) who renounce...",1.967992,Filmfour,"France,Germany,India,United Kingdom",2001-09-23,0.0,86.0,Released,NaN,The Warrior,6.3,15.0,NaN,हिन्दी
17229,0,Drama,25541,da,Former Danish servicemen Lars and Jimmy are th...,2.587911,NaN,"Sweden,Denmark",2009-10-21,0.0,90.0,Released,NaN,Brotherhood,7.1,21.0,NaN,Dansk


Ahora, se procede verificar aquellas filas cuyos id son iguales pero difieren en los datos de 1 o 2 columnas.

Como las columnas donde difieren los datos (```popularity```, ```vote_count```) no son columnas con un impacto menor en los datos, lo mas conveniente seria hacer una investigación adicional (Revisión de documentación) para determinar cuál es la entrada correcta o si ambas entradas son válidas.

Es posible que las diferencias se deban a cambios en la popularidad y el número de votos a lo largo del tiempo, especialmente si las filas corresponden a diferentes momentos en la vida útil de la película (por ejemplo, antes y después de su lanzamiento).

Sin embargo, dado el alcance del proyecto resulta difícil determinar con certeza si las diferencias en la popularidad y el número de votos de dos filas con el mismo id en un conjunto de datos de películas se deben a cambios reales en los datos o a errores de entrada de datos.

Por lo que la accion a tomar sera la siguiente: como no podemos elegir una de las filas como la "correcta", se procede a fusionar las dos filas en una sola sacando la mediana de los datos que difieren y usandola como sustitucion.

In [112]:
# Verificar para el mismo id hay datos que difieren
sustitucion_132641 = movies[movies['id'] == 132641][['id','popularity']]
sustitucion_132641

,id,popularity
838,132641,0.096079
30001,132641,0.619388


In [113]:
# Calcular la mediana de la columna "popularity" para filas con el id 132641
new_value = round(sustitucion_132641.groupby('id')['popularity'].median().iloc[0], 6)
# Actualizar el valor de la fila en la columna "popularity"
row_index = 838
drop_index = 30001
movies.loc[row_index, 'popularity'] = new_value
# Eliminar la fila con el indice correspondiente a drop_index
movies.drop(labels = drop_index, inplace = True)
# Verificar que los cambios se han realizado
movies[movies['id'] == 132641]

,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count,collection,languages
838,0,Drama,132641,ja,"Ten years into a marriage, the wife is disappo...",0.357734,Toho Company,Japan,1953-04-29,0.0,89.0,Released,NaN,Wife,0.0,0.0,NaN,日本語


In [114]:
# Verificar para el mismo id hay datos que difieren
sustitucion_10991 = movies[movies['id'] == 10991][['id','popularity','vote_count']]
sustitucion_10991

,id,popularity,vote_count
4114,10991,10.264597,143.0
44821,10991,6.480376,144.0


In [115]:
# Calcular la mediana de la columna "popularity" y "vote_count" para filas con el id 10991
new_value = round(sustitucion_10991.groupby('id')['popularity'].median().iloc[0], 6)
new_value_1 = round(sustitucion_10991.groupby('id')['vote_count'].median().iloc[0], 1)
# Actualizar el valor de la fila en la columna "popularity" y "vote_count"
row_index = 4114
drop_index = 44821
movies.loc[row_index, 'popularity'] = new_value
movies.loc[row_index, 'vote_count'] = new_value_1
# Eliminar la fila con el indice correspondiente a drop_index
movies.drop(labels = drop_index, inplace = True)
# Verificar que los cambios se han realizado
movies[movies['id'] == 10991]

,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count,collection,languages
4114,16000000,"Adventure,Fantasy,Animation,Action,Family",10991,ja,When Molly Hale's sadness of her father's disa...,8.372487,"TV Tokyo,4 Kids Entertainment,Nintendo,Pikachu...",Japan,2000-07-08,68411275.0,93.0,Released,Pokémon: Spell of the Unknown,Pokémon: Spell of the Unknown,6.0,143.5,Pokémon Collection,English


In [116]:
# Verificar para el mismo id hay datos que difieren
sustitucion_4912 = movies[movies['id'] == 4912][['id','popularity']]
sustitucion_4912

,id,popularity
5865,4912,11.331072
33826,4912,7.645827


In [117]:
# Calcular la mediana de la columna "popularity" para filas con el id 4912
new_value = round(sustitucion_4912.groupby('id')['popularity'].median().iloc[0], 6)
# Actualizar el valor de la fila en la columna "popularity"
row_index = 5865
drop_index = 33826
movies.loc[row_index, 'popularity'] = new_value
# Eliminar la fila con el indice correspondiente a drop_index
movies.drop(labels = drop_index, inplace = True)
# Verificar que los cambios se han realizado
movies[movies['id'] == 4912]

,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count,collection,languages
5865,30000000,"Comedy,Crime,Drama,Romance,Thriller",4912,en,"Television made him famous, but his biggest hi...",9.48845,"Miramax Films,Allied Filmmakers,Mad Chance",United States of America,2002-12-30,33013805.0,113.0,Released,Some things are better left top secret.,Confessions of a Dangerous Mind,6.6,281.0,NaN,English


In [118]:
# Verificar para el mismo id hay datos que difieren
sustitucion_15028 = movies[movies['id'] == 15028][['id','popularity', 'vote_count']]
sustitucion_15028

,id,popularity,vote_count
5130,15028,5.373623,89.0
33743,15028,4.920175,90.0


In [119]:
# Calcular la mediana de la columna "popularity" y "vote_count" para filas con el id 15028
new_value = round(sustitucion_15028.groupby('id')['popularity'].median().iloc[0], 6)
new_value_1 = round(sustitucion_15028.groupby('id')['vote_count'].median().iloc[0], 1)
# Actualizar el valor de la fila en la columna "popularity" y "vote_count"
row_index = 5130
drop_index = 33743
movies.loc[row_index, 'popularity'] = new_value
movies.loc[row_index, 'vote_count'] = new_value_1
# Eliminar la fila con el indice correspondiente a drop_index
movies.drop(labels = drop_index, inplace = True)
# Verificar que los cambios se han realizado
movies[movies['id'] == 15028]

,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count,collection,languages
5130,26000000,"Adventure,Family,Science Fiction,Thriller",15028,en,"Until now, Zak Gibbs' greatest challenge has b...",5.146899,"Paramount Pictures,Nickelodeon Movies,Valhalla...",United States of America,2002-03-17,38793283.0,94.0,Released,"The adventure of a lifetime, in a few mere sec...",Clockstoppers,4.9,89.5,NaN,"Český,English"


In [120]:
# Verificar para el mismo id hay datos que difieren
sustitucion_14788 = movies[movies['id'] == 14788][['id','popularity']]
sustitucion_14788

,id,popularity
10419,14788,3.185256
12066,14788,3.008299


In [121]:
# Calcular la mediana de la columna "popularity" para filas con el id 14788
new_value = round(sustitucion_14788.groupby('id')['popularity'].median().iloc[0], 6)
# Actualizar el valor de la fila en la columna "popularity"
row_index = 10419
drop_index = 12066
movies.loc[row_index, 'popularity'] = new_value
# Eliminar la fila con el indice correspondiente a drop_index
movies.drop(labels = drop_index, inplace = True)
# Verificar que los cambios se han realizado
movies[movies['id'] == 14788]

,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count,collection,languages
10419,1600000,"Drama,Crime,Mystery",14788,en,Set against the backdrop of a decaying Midwest...,3.096778,"Magnolia Pictures,Extension 765",United States of America,2005-09-03,0.0,73.0,Released,NaN,Bubble,6.4,36.0,NaN,English


In [122]:
# Verificar para el mismo id hay datos que difieren
sustitucion_84198 = movies[movies['id'] == 84198][['id','popularity']]
sustitucion_84198

,id,popularity
2564,84198,0.501046
21116,84198,1.673307


In [123]:
# Calcular la mediana de la columna "popularity" para filas con el id 84198
new_value = round(sustitucion_84198.groupby('id')['popularity'].median().iloc[0], 6)
# Actualizar el valor de la fila en la columna "popularity"
row_index = 2564
drop_index = 21116
movies.loc[row_index, 'popularity'] = new_value
# Eliminar la fila con el indice correspondiente a drop_index
movies.drop(labels = drop_index, inplace = True)
# Verificar que los cambios se han realizado
movies[movies['id'] == 84198]

,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count,collection,languages
2564,0,Documentary,84198,en,"Using personal stories, this powerful document...",1.087176,NaN,United States of America,2012-03-22,0.0,84.0,Released,One Nation. Underfed.,A Place at the Table,6.9,7.0,NaN,English


In [124]:
# Verificar para el mismo id hay datos que difieren
sustitucion_13209 = movies[movies['id'] == 13209][['id','popularity']]
sustitucion_13209

,id,popularity
11342,13209,1.528960
15765,13209,1.529879


In [125]:
# Calcular la mediana de la columna "popularity" para filas con el id 13209
new_value = round(sustitucion_13209.groupby('id')['popularity'].median().iloc[0], 6)
# Actualizar el valor de la fila en la columna "popularity"
row_index = 11342
drop_index = 15765
movies.loc[row_index, 'popularity'] = new_value
# Eliminar la fila con el indice correspondiente a drop_index
movies.drop(labels = drop_index, inplace = True)
# Verificar que los cambios se han realizado
movies[movies['id'] == 13209]

,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count,collection,languages
11342,2500,"Drama,Comedy,Foreign",13209,fa,"Since women are banned from soccer matches, Ir...",1.52942,Jafar Panahi Film Productions,Iran,2006-05-26,0.0,93.0,Released,NaN,Offside,6.7,27.0,NaN,فارسی


In [126]:
# Verificar para el mismo id hay datos que difieren
sustitucion_77221 = movies[movies['id'] == 77221][['id','popularity']]
sustitucion_77221

,id,popularity
11155,77221,6.652197
20843,77221,6.475665


In [127]:
# Calcular la mediana de la columna "popularity" para filas con el id 77221
new_value = round(sustitucion_77221.groupby('id')['popularity'].median().iloc[0], 6)
# Actualizar el valor de la fila en la columna "popularity"
row_index = 11155
drop_index = 20843
movies.loc[row_index, 'popularity'] = new_value
# Eliminar la fila con el indice correspondiente a drop_index
movies.drop(labels = drop_index, inplace = True)
# Verificar que los cambios se han realizado
movies[movies['id'] == 77221]

,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count,collection,languages
11155,40000000,"Adventure,Drama",77221,en,"On the Arabian Peninsula in the 1930s, two war...",6.563931,"France 2 Cinéma,Quinta Communications,Carthago...","France,Italy,Qatar,Tunisia",2011-12-21,5446000.0,130.0,Released,NaN,Black Gold,5.9,77.0,NaN,English


In [128]:
# Verificar para el mismo id hay datos que difieren
sustitucion_109962 = movies[movies['id'] == 109962][['id','popularity']]
sustitucion_109962

,id,popularity
5710,109962,12.180836
20899,109962,10.396878


In [129]:
# Calcular la mediana de la columna "popularity" para filas con el id 109962
new_value = round(sustitucion_109962.groupby('id')['popularity'].median().iloc[0], 6)
# Actualizar el valor de la fila en la columna "popularity"
row_index = 5710
drop_index = 20899
movies.loc[row_index, 'popularity'] = new_value
# Eliminar la fila con el indice correspondiente a drop_index
movies.drop(labels = drop_index, inplace = True)
# Verificar que los cambios se han realizado
movies[movies['id'] == 109962]

,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count,collection,languages
5710,0,Drama,109962,en,Two literary women compete for 20 years: one w...,11.288857,"Metro-Goldwyn-Mayer (MGM),Jaquet",United States of America,1981-09-23,0.0,115.0,Released,"From the very beginning, they knew they'd be f...",Rich and Famous,4.9,7.0,NaN,English


In [130]:
# Verificar para el mismo id hay datos que difieren
sustitucion_22649 = movies[movies['id'] == 22649][['id','popularity']]
sustitucion_22649

,id,popularity
949,22649,1.914697
15074,22649,2.411191


In [131]:
# Calcular la mediana de la columna "popularity" para filas con el id 22649
new_value = round(sustitucion_22649.groupby('id')['popularity'].median().iloc[0], 6)
# Actualizar el valor de la fila en la columna "popularity"
row_index = 949
drop_index = 15074
movies.loc[row_index, 'popularity'] = new_value
# Eliminar la fila con el indice correspondiente a drop_index
movies.drop(labels = drop_index, inplace = True)
# Verificar que los cambios se han realizado
movies[movies['id'] == 22649]

,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count,collection,languages
949,4,"Drama,Romance,War",22649,en,British nurse Catherine Barkley (Helen Hayes) ...,2.162944,Paramount Pictures,United States of America,1932-12-08,25.0,89.0,Released,Every woman who has loved will understand,A Farewell to Arms,6.2,29.0,NaN,English


In [132]:
# Verificar para el mismo id hay datos que difieren
sustitucion_110428 = movies[movies['id'] == 110428][['id','popularity']]
sustitucion_110428

,id,popularity
4356,110428,0.134014
23534,110428,0.110065


In [133]:
# Calcular la mediana de la columna "popularity" para filas con el id 110428
new_value = round(sustitucion_110428.groupby('id')['popularity'].median().iloc[0], 6)
# Actualizar el valor de la fila en la columna "popularity"
row_index = 4356
drop_index = 23534
movies.loc[row_index, 'popularity'] = new_value
# Eliminar la fila con el indice correspondiente a drop_index
movies.drop(labels = drop_index, inplace = True)
# Verificar que los cambios se han realizado
movies[movies['id'] == 110428]

,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count,collection,languages
4356,3512454,Drama,110428,fr,"Winter, 1915. Confined by her family to an asy...",0.12204,"Canal+,Arte France Cinéma,3B Productions,C.R.R...",France,2013-03-13,115860.0,95.0,Released,NaN,Camille Claudel 1915,7.0,20.0,NaN,Français


In [134]:
# Verificar para el mismo id hay datos que difieren
sustitucion_69234 = movies[movies['id'] == 69234][['id','popularity']]
sustitucion_69234

,id,popularity
9576,69234,0.441872
26625,69234,0.438490


In [135]:
# Calcular la mediana de la columna "popularity" para filas con el id 69234
new_value = round(sustitucion_69234.groupby('id')['popularity'].median().iloc[0], 6)
# Actualizar el valor de la fila en la columna "popularity"
row_index = 9576
drop_index = 26625
movies.loc[row_index, 'popularity'] = new_value
# Eliminar la fila con el indice correspondiente a drop_index
movies.drop(labels = drop_index, inplace = True)
# Verificar que los cambios se han realizado
movies[movies['id'] == 69234]

,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count,collection,languages
9576,10000000,"Drama,Horror,Music,Romance,TV Movie",69234,en,Count de Chagnie has discovered Christine's si...,0.440181,"Beta Film,Reteitalia,TF1,Hexatel,Saban/Scheric...","France,Germany,Italy,United States of America",1990-03-18,0.0,168.0,Released,NaN,The Phantom of the Opera,5.0,3.0,NaN,"English,Italiano"


In [136]:
# Verificar para el mismo id hay datos que difieren
sustitucion_12600 = movies[movies['id'] == 12600][['id','popularity']]
sustitucion_12600

,id,popularity
5535,12600,7.072301
44826,12600,6.080108


In [137]:
# Calcular la mediana de la columna "popularity" para filas con el id 12600
new_value = round(sustitucion_12600.groupby('id')['popularity'].median().iloc[0], 6)
# Actualizar el valor de la fila en la columna "popularity"
row_index = 5535
drop_index = 44826
movies.loc[row_index, 'popularity'] = new_value
# Eliminar la fila con el indice correspondiente a drop_index
movies.drop(labels = drop_index, inplace = True)
# Verificar que los cambios se han realizado
movies[movies['id'] == 12600]

,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count,collection,languages
5535,0,"Adventure,Fantasy,Animation,Science Fiction,Fa...",12600,ja,"All your favorite Pokémon characters are back,...",6.576204,NaN,"Japan,United States of America",2001-07-06,28023563.0,75.0,Released,NaN,Pokémon 4Ever: Celebi - Voice of the Forest,5.7,82.0,Pokémon Collection,日本語


Confirmando si los cambios se realizaron y ya no hay id duplicados

In [138]:
# Verificando la cantiad de duplicados en la columna id
movies[movies.duplicated(subset = 'id', keep = False)].shape[0]

0

## **Unión de credits y movies**

### Verificando la similitud de los id antes de la union

Dado que el id en el dataframe credits y movies, es el mismo ya que corresponde al id de cada pelicula. Se procede a unificar ambos dataframes para facilitar el resto de la limpieza.

In [ ]:
# Cantidad de id en cada df
print('Cantidad de id en el dataframe credicts: ', credits.id.shape[0])
print('Cantidad de id en el dataframe movies: ', movies.id.shape[0])

Cantidad de id en el dataframe credicts:  45432
Cantidad de id en el dataframe movies:  45433


In [ ]:
# Verificar datos duplicados
print('Id duplicados en el dataframe credicts: ', movies[movies.duplicated(subset = 'id', keep = False)].shape[0])
print('Id duplicados en el dataframe movies: ', movies[movies.duplicated(subset = 'id', keep = False)].shape[0])

Id duplicados en el dataframe credicts:  0
Id duplicados en el dataframe movies:  0


In [ ]:
# Comparar los id entre ambos dataframes
movies_test = movies[['id']]
credits_test = credits[['id']]

comparacion_id = movies_test.merge(credits_test, how = 'outer', indicator = 'union')
list_both = np.array([comparacion_id.union == 'both']).sum()
list_left_only = np.array([comparacion_id.union == 'left_only']).sum()
list_right_only = np.array([comparacion_id.union == 'right_only']).sum()

print(f'Cantidad de Id presentes en ambos: {list_both}')
print(f'Cantidad de Id que estan presentes en movies pero no en credits: {list_left_only}')
print(f'Cantidad de Id que estan presentes en credits pero no en movies: {list_right_only}')

Cantidad de Id presentes en ambos: 45432
Cantidad de Id que estan presentes en movies pero no en credits: 1
Cantidad de Id que estan presentes en credits pero no en movies: 0


In [ ]:
# Encontrar el Id que está solo en movies
id_movies_only = comparacion_id[comparacion_id['union'] == 'left_only']['id']
print(f'El Id que está presente solo en movies es: {id_movies_only.values[0]}')

El Id que está presente solo en movies es: 401840


In [ ]:
# Verificar si el id 401840 no se encuentra en el dataframe 'credits_csv' en el cual la data se encuentra sin modificar
credits_csv[credits_csv['id'] == 401840]

,cast,crew,id


In [ ]:
# Verificar a que pelicula corresponde el id 401840
movies[movies['id'] == 401840][['id', 'title', 'popularity', 'release_date']]

,id,title,popularity,release_date
42883,401840,School's out,0.207775,2017-05-30


Pese a que hay un id que no esta presente en en ambos Dataframes, se procede a unirlos utilizando la columna id como clave de unión.

Para esto se usara el metodo merge() de Pandas, con el argumento how establecido en 'outer' el cual incluirá todas las filas de ambos DataFrames, incluso si no tienen una correspondencia (el id faltante en credits). Las filas que no tienen una correspondencia se completarán con valores NaN.

In [139]:
# Union de movies y credits
merged_df = movies.merge(credits, on = 'id', how = 'outer')
merged_df.head()

,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count,collection,languages,director,actors
0,30000000,"Animation,Comedy,Family",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,Pixar Animation Studios,United States of America,1995-10-30,373554033.0,81.0,Released,NaN,Toy Story,7.7,5415.0,Toy Story Collection,English,John Lasseter,"Tom Hanks,Tim Allen,Don Rickles,Jim Varney,Wal..."
1,65000000,"Adventure,Fantasy,Family",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"TriStar Pictures,Teitler Film,Interscope Commu...",United States of America,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,NaN,"English,Français",Joe Johnston,"Robin Williams,Jonathan Hyde,Kirsten Dunst,Bra..."
2,0,"Romance,Comedy",15602,en,A family wedding reignites the ancient feud be...,11.712900,"Warner Bros.,Lancaster Gate",United States of America,1995-12-22,0.0,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,Grumpy Old Men Collection,English,Howard Deutch,"Walter Matthau,Jack Lemmon,Ann-Margret,Sophia ..."
3,16000000,"Comedy,Drama,Romance",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,Twentieth Century Fox Film Corporation,United States of America,1995-12-22,81452156.0,127.0,Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,NaN,English,Forest Whitaker,"Whitney Houston,Angela Bassett,Loretta Devine,..."
4,0,Comedy,11862,en,Just when George Banks has recovered from his ...,8.387519,"Sandollar Productions,Touchstone Pictures",United States of America,1995-02-10,76578911.0,106.0,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,Father of the Bride Collection,English,Charles Shyer,"Steve Martin,Diane Keaton,Martin Short,Kimberl..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45428,0,"Drama,Family",439050,fa,Rising and falling between a man and woman.,0.072051,NaN,Iran,NaT,0.0,90.0,Released,Rising and falling between a man and woman,Subdue,4.0,1.0,NaN,فارسی,Hamid Nematollah,"Leila Hatami,Kourosh Tahami,Elham Korda"
45429,0,Drama,111109,tl,An artist struggles to finish his work while a...,0.178241,Sine Olivia,Philippines,2011-11-17,0.0,360.0,Released,NaN,Century of Birthing,9.0,3.0,NaN,,Lav Diaz,"Angel Aquino,Perry Dizon,Hazel Orencio,Joel To..."
45430,0,"Action,Drama,Thriller",67758,en,"When one of her hits goes wrong, a professiona...",0.903007,American World Pictures,United States of America,2003-08-01,0.0,90.0,Released,A deadly game of wits.,Betrayal,3.8,6.0,NaN,English,Mark L. Lester,"Erika Eleniak,Adam Baldwin,Julie du Page,James..."
45431,0,NaN,227506,en,"In a small town live two brothers, one a minis...",0.003503,Yermoliev,Russia,1917-10-21,0.0,87.0,Released,NaN,Satan Triumphant,0.0,0.0,NaN,NaN,Yakov Protazanov,"Iwan Mosschuchin,Nathalie Lissenko,Pavel Pavlo..."


In [143]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45433 entries, 0 to 45432
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   budget                45433 non-null  int32         
 1   genres                42991 non-null  object        
 2   id                    45433 non-null  int32         
 3   original_language     45422 non-null  object        
 4   overview              44479 non-null  object        
 5   popularity            45430 non-null  float64       
 6   production_companies  33562 non-null  object        
 7   production_countries  39151 non-null  object        
 8   release_date          45346 non-null  datetime64[ns]
 9   revenue               45430 non-null  float64       
 10  runtime               45173 non-null  float64       
 11  status                45349 non-null  object        
 12  tagline               20401 non-null  object        
 13  title           

In [140]:
merged_df.shape

(45433, 20)

### Datos faltantes

¿cuántos datos faltantes hay y que % representan en base al total de datos?

In [144]:
# Cantidad de datos faltantes por columna
missing_values_count = merged_df.isnull().sum()
columns = merged_df.shape[1]
missing_values_count[0:columns]

budget                      0
genres                   2442
id                          0
original_language          11
overview                  954
popularity                  3
production_companies    11871
production_countries     6282
release_date               87
revenue                     3
runtime                   260
status                     84
tagline                 25032
title                       3
vote_average                3
vote_count                  3
collection              40945
languages                3830
director                  888
actors                   2415
dtype: int64

In [145]:
# % de datos faltantes respecto al total de los datos
total_cells = np.product(merged_df.shape)
total_missing = missing_values_count.sum()
percent_missing = (total_missing / total_cells) * 100
round(percent_missing, 2)

10.47

Todas las columnas excepto una presentan datos faltantes, sin embargo, las que mayor datos nulos presentan son: ```collection```, ```tagline``` y ```production_companies```.

#### Atributo Collection



In [146]:
total_collection = merged_df['collection'].shape[0]
total_missing_collection = merged_df['collection'].isnull().sum()
percent_missing_collection = (total_missing_collection / total_collection) * 100
value = round(percent_missing_collection, 2)
print(f'De {total_collection} registros que posee la columna collection, {total_missing_collection} corresponden a los datos faltantes. Lo que representa el {value}% de los datos de la columna')

De 45433 registros que posee la columna collection, 40945 corresponden a los datos faltantes. Lo que representa el 90.12% de los datos de la columna


Considerando que la columna tiene el 90.12% de sus registros como nulos y no esta aportando información significativa. Se procede a descartarla

In [148]:
merged_df = merged_df.drop(['collection'], axis = 1)
merged_df.columns

Index(['budget', 'genres', 'id', 'original_language', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'languages', 'director', 'actors'],
      dtype='object')

In [149]:
# Cantidad de datos faltantes considerando la eliminancion del atributo collection
missing_values_count = merged_df.isnull().sum()

# % de datos faltantes respecto al total de los datos
total_cells = np.product(merged_df.shape)
total_missing = missing_values_count.sum()
percent_missing = (total_missing / total_cells) * 100
round(percent_missing, 2)

6.28

Con el descarte de la columna collection, la cantidad de datos faltantes disminuyo un 4.19%

#### Atributo release_date

In [151]:
total_release_date = merged_df['release_date'].shape[0]
total_missing_release_date = merged_df['release_date'].isnull().sum()
percent_missing_release_date = (total_missing_release_date / total_release_date) * 100
value = round(percent_missing_release_date, 2)
print(f'De {total_release_date} registros que posee la columna release_date, {total_missing_release_date} corresponden a los datos faltantes. Lo que representa el {value}% de los datos de la columna')

De 45433 registros que posee la columna release_date, 87 corresponden a los datos faltantes. Lo que representa el 0.19% de los datos de la columna


In [176]:
null_dates = merged_df[merged_df['release_date'].isnull()]
null_dates.head(3)

,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count,languages,director,actors
711,0,NaN,365371,en,Seven New Zealand women speak about their live...,0.005625,NaN,NaN,NaT,0.0,95.0,Released,NaN,War Stories Our Mother Never Told Us,0.0,0.0,NaN,NaN,NaN
734,0,NaN,215107,en,Vermont is for Lovers is an independently prod...,0.750000,NaN,NaN,NaT,0.0,88.0,Released,NaN,Vermont Is for Lovers,0.0,0.0,NaN,John O'Brien,"George Thrush,Marya Cohn,Ann O'Brien"
3459,0,Drama,94214,en,"Jails, Hospitals &amp; Hip-Hop is a cinematic ...",0.009057,NaN,NaN,NaT,10.0,90.0,NaN,three worlds / two million voices / one genera...,"Jails, Hospitals & Hip-Hop",0.0,0.0,NaN,NaN,NaN


In [174]:
print('De las 87 filas con datos nulos en release_date:')
print('El atributo director tiene:', null_dates['director'].isnull().sum(), 'datos nulos')
print('El atributo actors tiene:', null_dates['actors'].isnull().sum(), 'datos nulos')

De las 87 filas con datos nulos en release_date:
El atributo director tiene:  52 datos nulos
El atributo actors tiene:  66 datos nulos


In [175]:
# Cantidad de valores no nulos en una determinada porcion de filas
null_dates.notna().sum(axis = 1).value_counts()

11    24
12    17
13    15
15     9
9      7
14     4
10     3
7      3
17     3
16     2
dtype: int64

El resultado anterior nos indica que: De las 87 filas donde release_date tiene datos nulos, solo 3 de ellas tienen 17 valores no nulos teniendo en cuenta que el total de columna es 19.

Por consiguiente, en la siguiente lineas de codigo se efectua la eliminacion de estos datos faltantes que solo representan el 0.19% de la columna release_data, considerando ademas que de los 87 filas 52 tiene datos nulos para el atributo Director y 66 para el atributo actors.

In [177]:
# Eliminar las filas donde la columna release_date tiene valores nulos
merged_df = merged_df.dropna(subset=['release_date'])
# Comprobar que ya no hay datos nulos en release_date
merged_df['release_date'].isnull().sum()

0

In [178]:
# Cantidad de datos faltantes considerando la eliminancion de los valores nulos en release_date
missing_values_count = merged_df.isnull().sum()

# % de datos faltantes respecto al total de los datos
total_cells = np.product(merged_df.shape)
total_missing = missing_values_count.sum()
percent_missing = (total_missing / total_cells) * 100
round(percent_missing, 2)

6.22

Con la eliminacion de los valores nulos en el campo release_date, la cantidad de datos faltantes disminuyo un 0.06%

### Atributos revenue y budget

A partir de estos dos atributos se creara la columna de retorno de inversion llamada ```return```. Por lo que primero se verificara que no hayan valores nulos

#### Atributo revenue

In [182]:
# Cantidad de valores nulos en la columna revenue
merged_df.revenue.isnull().sum()

0

En el DataFrame 'movies_csv', en su estado original, sólo había 6 valores faltantes en la columna 'revenue'. Sin embargo, después de eliminar 3 filas con datos desfasados que impedían el cambio de tipo de datos en varios campos, la cantidad de valores faltantes se redujo a 3. Posteriormente, en el DataFrame 'merged_df', sólo quedaron 3 valores faltantes en la columna 'revenue', los cuales fueron eliminados al eliminar los valores nulos en el campo 'release_date', resultando en un conjunto completo de datos para la columna 'revenue'.

#### Atributo budget

In [184]:
# Cantidad de valores nulos en la columna budget
merged_df.budget.isnull().sum()

0

El Dataframe de movies_csv en su data cruda no presento datos faltantes para este campo.

### Anadiendo atributos

#### Atributo release_year

In [187]:
# Crear una nueva columna 'release_year' a partir de la columna 'release_date'
merged_df['release_year'] = merged_df['release_date'].dt.year
merged_df['release_year'].head()

0        1995
1        1995
2        1995
3        1995
4        1995
         ... 
45427    1991
45429    2011
45430    2003
45431    1917
45432    2017
Name: release_year, Length: 45346, dtype: int64

#### Atributo return

La columna return tendrá los datos correspondientes al retorno de inversión para cada película. Esto se calculará diviendo los campos revenue y budget (```revenue```/ ```budget```), como hay una división verifiquemos primero si hay valores igual a 0.

In [188]:
# Cantidad de registros con valor igual a 0
np.array([merged_df['budget'] == 0]).sum()

36470

In [190]:
# Cantidad de registros con valor igual a 0
np.array([merged_df['revenue'] == 0]).sum()

37949

Como hay registros con valores igual a 0 para ambos campos, la división revenue / budget generará un error de división por cero. Para evitar esto, se decide crea una función que verifique si el valor de 'budget' es 0, y si es así, que devuelva 0. De lo contrario, la función realizará la división.

In [191]:
# Función que calcula el retorno de inversión y maneja los valores de budget igual a 0
def calculate_return(row):
  if row['budget'] == 0:
    return 0
  else:
    return row['revenue'] / row['budget']

print('Funcion creada')

Funcion creada


In [192]:
# Se crea la columna 'return' utilizando la función 'calculate_return'
merged_df['return'] = merged_df.apply(calculate_return, axis = 1)

# Reemplaza los valores faltantes con 0
merged_df['return'] = merged_df['return'].fillna(0)

merged_df['return'].head()

# **Carga / Disponibilización de datos**

In [193]:
merged_df.head()

,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,...,status,tagline,title,vote_average,vote_count,languages,director,actors,release_year,return
0,30000000,"Animation,Comedy,Family",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,Pixar Animation Studios,United States of America,1995-10-30,373554033.0,...,Released,NaN,Toy Story,7.7,5415.0,English,John Lasseter,"Tom Hanks,Tim Allen,Don Rickles,Jim Varney,Wal...",1995,12.451801
1,65000000,"Adventure,Fantasy,Family",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"TriStar Pictures,Teitler Film,Interscope Commu...",United States of America,1995-12-15,262797249.0,...,Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,"English,Français",Joe Johnston,"Robin Williams,Jonathan Hyde,Kirsten Dunst,Bra...",1995,4.043035
2,0,"Romance,Comedy",15602,en,A family wedding reignites the ancient feud be...,11.712900,"Warner Bros.,Lancaster Gate",United States of America,1995-12-22,0.0,...,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,English,Howard Deutch,"Walter Matthau,Jack Lemmon,Ann-Margret,Sophia ...",1995,0.000000
3,16000000,"Comedy,Drama,Romance",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,Twentieth Century Fox Film Corporation,United States of America,1995-12-22,81452156.0,...,Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,English,Forest Whitaker,"Whitney Houston,Angela Bassett,Loretta Devine,...",1995,5.090760
4,0,Comedy,11862,en,Just when George Banks has recovered from his ...,8.387519,"Sandollar Productions,Touchstone Pictures",United States of America,1995-02-10,76578911.0,...,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,English,Charles Shyer,"Steve Martin,Diane Keaton,Martin Short,Kimberl...",1995,0.000000


Ahora los datos estan limpios y se pueden exportar para luego ser utilizados para analisis futuros.

In [194]:
# merged_df.to_csv('movies_clean.csv', index = False)